In [5]:
import csv
import cv2
import numpy as np

def loadImage(line):
    xs = []
    ys = []
    
    correction = 0.22
    
    for view in range(3):
        origPath = line[view]
        filename = origPath.split('/')[-1]
        newPath = "data/IMG/" + filename
        
        x = cv2.imread(newPath)
        x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
        
        steer = float(line[3])
                       
        if view == 1: # Left view
            steer += correction
        if view == 2: # Right view
            steer -= correction
        
        xs.append(np.array(x))
        ys.append(steer)
        
    
    return xs,ys

# load training file list
lines = []
with open('data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)
        
# load images and steering angle
images = []
measurements = []
for line in lines:
    xs, ys = loadImage(line)
    for i in range(len(xs)):
        x = xs[i]
        y = ys[i]
        images.append(x)
        measurements.append(y)
        images.append(cv2.flip(x, 1))
        measurements.append(-1*y)
    
X_train = np.array(images)
y_train = np.array(measurements)

img_shape = X_train[0].shape

print('X size:', X_train.shape)
print('y size:', y_train.shape)
print('img sample size:', img_shape)

X size: (39660, 160, 320, 3)
y size: (39660,)
img sample size: (160, 320, 3)


In [4]:
from keras.models import Sequential
from keras.layers import Input, Flatten, Dense, Lambda, Convolution2D,MaxPooling2D, Dropout, Cropping2D
from keras.optimizers import Adam

img_shape = (64,64,3)

def AlexNet():
    model = Sequential()
    model.add(Cropping2D(cropping=((top_crop,bottom_crop), (0,0)), input_shape=img_shape, dim_ordering='tf'))
    model.add(Lambda(lambda x:x/255.0-0.5))
    model.add(Convolution2D(6,5,5,activation="relu"))
    model.add(MaxPooling2D())
    model.add(Convolution2D(6,5,5,activation="relu"))
    model.add(MaxPooling2D())
    model.add(Flatten())
    model.add(Dense(120))
    model.add(Dropout(0.2))
    model.add(Dense(84))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    return model

def NvidiaNet():
    model = Sequential()
    model.add(Cropping2D(cropping=((top_crop,bottom_crop), (0,0)), input_shape=img_shape, dim_ordering='tf'))
    model.add(Lambda(lambda x:x/255.0-0.5))
    model.add(Convolution2D(24,5,5, subsample=(2,2),activation="relu"))
    model.add(Convolution2D(36,5,5, subsample=(2,2),activation="relu"))
    model.add(Convolution2D(48,5,5, subsample=(2,2),activation="relu"))
    model.add(Convolution2D(64,3,3, activation="relu"))
    model.add(Convolution2D(64,3,3, activation="relu"))
    model.add(Flatten())
    model.add(Dropout(0.2))
    model.add(Dense(100))
    model.add(Dropout(0.2))
    model.add(Dense(50))
    model.add(Dense(10))
    model.add(Dense(1))
    return model

model = NvidiaNet()
adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='mse', optimizer=adam)
history_object = model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=10, batch_size=256, verbose=1)

# model.save('model_center_andLeftRight_cropped_TwoTracks_AlexNet.h5')
model.save('model_center_andLeftRight_cropped_TwoTracks_NvidiaNet.h5')


Train on 31728 samples, validate on 7932 samples
Epoch 1/10
31728/31728 [==============================] - 12s - loss: 0.0946 - val_loss: 0.1665
Epoch 2/10
31728/31728 [==============================] - 10s - loss: 0.0738 - val_loss: 0.1360
Epoch 3/10
31728/31728 [==============================] - 10s - loss: 0.0651 - val_loss: 0.1419
Epoch 4/10
31728/31728 [==============================] - 10s - loss: 0.0571 - val_loss: 0.1475
Epoch 5/10
31728/31728 [==============================] - 10s - loss: 0.0495 - val_loss: 0.1468
Epoch 6/10
31728/31728 [==============================] - 10s - loss: 0.0435 - val_loss: 0.1548
Epoch 7/10
31728/31728 [==============================] - 10s - loss: 0.0387 - val_loss: 0.1615
Epoch 8/10
31728/31728 [==============================] - 10s - loss: 0.0354 - val_loss: 0.1573
Epoch 9/10
31728/31728 [==============================] - 10s - loss: 0.0331 - val_loss: 0.1553
Epoch 10/10
31728/31728 [==============================] - 10s - loss: 0.0302 - val_los

In [ ]:
from keras.models import Model
import matplotlib.pyplot as plt

### print the keys contained in the history object
print(history_object.history.keys())

### plot the training and validation loss for each epoch
plt.plot(history_object.history['loss'])
plt.plot(history_object.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.show()